# Converting the CSV files to FITS files that can be read by FastSpecFit

FastSpecFit is currently only able to ingest spectra that are on as wavelength grid with equal wavelength bins. So aside from just reformatting the files I also need to resample all the spectra. Currently this is done using a linear interpolation on a new wavelength grid.

In [1]:
import glob
import numpy as np
from astropy.table import Table
import fitsio
import sys

sys.path.insert(0,'/global/u2/d/dscholte/forked_packages/desigal/py')
import desigal.specutils
from desigal.specutils.resample import linear_resample

In [2]:
spectra_files = glob.glob('Spectra*.csv')

In [18]:
def convert_spectra_file(filename, new_wave):
    table = Table.read(filename)
    nobj = int((len(table.colnames)-1)/3)
    print(table.colnames)
    flux = np.zeros((nobj, len(new_wave)))
    ivar = np.zeros((nobj, len(new_wave)))
    for i in range(nobj):
        fl, iv = linear_resample(
            new_wave, 
            np.expand_dims(table['Wavelength_radius_'+str(i+1)].data, axis=0), 
            np.expand_dims(table['flux_radius_'+str(i+1)].data, axis=0), 
            ivar=np.expand_dims(table['ivar_radius_'+str(i+1)].data, axis=0)
        )
        flux[i] = fl[0]
        ivar[i] = iv[0]
    mask = np.isnan(flux)|np.isnan(ivar)|np.isinf(flux)|np.isinf(ivar)
    flux[mask] = 0.0
    ivar[mask] = 0.0

    desigal.specutils.stack.write_binned_stacks(
        filename[:-3]+'fits', 
        new_wave,
        flux,
        ivar,
        stackids = np.arange(1,nobj+1),
        table_column_dict = {},
        table_format_dict = {},
    )
    return

In [19]:
new_wave = np.arange(3550,9500,1) # 1 Angstrom wavelength bins
[convert_spectra_file(filename, new_wave) for filename in spectra_files]

['col0', 'flux_radius_1', 'ivar_radius_1', 'Wavelength_radius_1', 'flux_radius_2', 'ivar_radius_2', 'Wavelength_radius_2', 'flux_radius_3', 'ivar_radius_3', 'Wavelength_radius_3', 'flux_radius_4', 'ivar_radius_4', 'Wavelength_radius_4', 'flux_radius_5', 'ivar_radius_5', 'Wavelength_radius_5']
['col0', 'flux_radius_1', 'ivar_radius_1', 'Wavelength_radius_1', 'flux_radius_2', 'ivar_radius_2', 'Wavelength_radius_2', 'flux_radius_3', 'ivar_radius_3', 'Wavelength_radius_3', 'flux_radius_4', 'ivar_radius_4', 'Wavelength_radius_4', 'flux_radius_5', 'ivar_radius_5', 'Wavelength_radius_5', 'flux_radius_6', 'ivar_radius_6', 'Wavelength_radius_6', 'flux_radius_7', 'ivar_radius_7', 'Wavelength_radius_7']
['col0', 'flux_radius_1', 'ivar_radius_1', 'Wavelength_radius_1', 'flux_radius_2', 'ivar_radius_2', 'Wavelength_radius_2', 'flux_radius_3', 'ivar_radius_3', 'Wavelength_radius_3', 'flux_radius_4', 'ivar_radius_4', 'Wavelength_radius_4', 'flux_radius_5', 'ivar_radius_5', 'Wavelength_radius_5', 'flu

[None, None, None, None, None]

In [20]:
spectra_files

['Spectra_stacks_Nov_8996-12701.csv',
 'Spectra_stacks_Nov_9042-9101.csv',
 'Spectra_stacks_Nov_8241-9101.csv',
 'Spectra_stacks_Nov_8144-6104.csv',
 'Spectra_stacks_Nov_8600-12702.csv']

In [16]:
table = Table.read(spectra_files[2])
nobj = int((len(table.colnames)-1)/3)
print(table.colnames)
flux = np.zeros((nobj, len(new_wave)))
ivar = np.zeros((nobj, len(new_wave)))
'''
for i in range(nobj):
        fl, iv = linear_resample(
            new_wave, 
            np.expand_dims(table['Wavelength_radius_'+str(i+1)].data, axis=0), 
            np.expand_dims(table['flux_radius_'+str(i+1)].data, axis=0), 
            ivar=np.expand_dims(table['ivar_radius_'+str(i+1)].data, axis=0)
        )
        flux[i] = fl[0]
        ivar[i] = iv[0]
    mask = np.isnan(flux)|np.isnan(ivar)
    flux[mask] = 0.0
    ivar[mask] = 0.0

    desigal.specutils.stack.write_binned_stacks(
        filename[:-3]+'fits', 
        new_wave,
        flux,
        ivar,
        stackids = np.arange(1,nobj+1),
        table_column_dict = {},
        table_format_dict = {},
'''

['col0', 'flux_radius_1', 'ivar_radius_1', 'Wavelength_radius_1', 'flux_radius_2', 'ivar_radius_2', 'Wavelength_radius_2', 'flux_radius_3', 'ivar_radius_3', 'Wavelength_radius_3', 'flux_radius_4', 'ivar_radius_4', 'Wavelength_radius_4', 'flux_radius_5', 'ivar_radius_5', 'Wavelength_radius_5', 'flux_radius_6', 'ivar_radius_6', 'Wavelength_radius_6']


"\nfor i in range(nobj):\n        fl, iv = linear_resample(\n            new_wave, \n            np.expand_dims(table['Wavelength_radius_'+str(i+1)].data, axis=0), \n            np.expand_dims(table['flux_radius_'+str(i+1)].data, axis=0), \n            ivar=np.expand_dims(table['ivar_radius_'+str(i+1)].data, axis=0)\n        )\n        flux[i] = fl[0]\n        ivar[i] = iv[0]\n    mask = np.isnan(flux)|np.isnan(ivar)\n    flux[mask] = 0.0\n    ivar[mask] = 0.0\n\n    desigal.specutils.stack.write_binned_stacks(\n        filename[:-3]+'fits', \n        new_wave,\n        flux,\n        ivar,\n        stackids = np.arange(1,nobj+1),\n        table_column_dict = {},\n        table_format_dict = {},\n"

In [17]:
table

col0,flux_radius_1,ivar_radius_1,Wavelength_radius_1,flux_radius_2,ivar_radius_2,Wavelength_radius_2,flux_radius_3,ivar_radius_3,Wavelength_radius_3,flux_radius_4,ivar_radius_4,Wavelength_radius_4,flux_radius_5,ivar_radius_5,Wavelength_radius_5,flux_radius_6,ivar_radius_6,Wavelength_radius_6
int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
0,4.083658695220947,0.4675237,3420.0952613148443,6.63925838470459,0.6092036,3420.6489295676624,12.270097732543945,0.8954938,3420.97932176263,16.212337493896484,1.0765283,3421.0819262921955,16.43755340576172,1.1689073,3420.77009886266,10.88021469116211,1.2616596,3420.799986281522
1,9.131622314453125,0.474043,3420.8828580233203,6.9237494468688965,0.60964745,3421.4366537776436,11.572415351867676,0.89083093,3421.767122056985,12.1144437789917,1.0667176,3421.8697502148366,11.542815208435059,1.1521065,3421.5578509761153,11.1364164352417,1.2597165,3421.5877452776017
2,12.029077529907227,0.5073133,3421.6706361035176,19.080078125,0.6437401,3422.224559388708,16.547325134277344,0.908677,3422.5551037699433,14.099601745605469,1.0601883,3422.6577555615227,19.778118133544922,1.1403599,3422.3457844970785,13.065722465515137,1.2499316,3422.375685682775
3,12.27873420715332,0.49249673,3422.4585955972098,15.92141056060791,0.62746704,3423.0126464426353,25.878620147705078,0.90001976,3423.343266943291,24.90515422821045,1.0570519,3423.44594237404,19.321243286132812,1.1323025,3423.1338994673324,16.676042556762695,1.2342384,3423.163807538824
4,9.307362079620361,0.45342994,3423.2467365461666,16.894866466522217,0.5909987,3423.800914981203,21.532793045043945,0.8665372,3424.1316116188073,20.09951400756836,1.0368078,3424.23431069417,18.111597061157227,1.1200626,3423.922195928655,20.421815872192383,1.2311523,3423.952110887528
5,8.186689376831055,0.45830014,3424.0350589921823,17.629939079284668,0.5968157,3424.589365046211,27.261664390563965,0.86805993,3424.9201378382973,23.024165153503418,1.027752,3425.0228605637185,22.99628734588623,1.1091653,3424.710673922849,23.07258892059326,1.2146173,3424.7405957706883
6,9.43792724609375,0.45782518,3424.823562977045,14.517984390258789,0.59838974,3425.3779966794546,27.613618850708008,0.87575257,3425.7088456435604,24.21684741973877,1.0307996,3425.811592024487,22.7177791595459,1.1038721,3425.49933349171,22.716205596923828,1.2152343,3425.5292622301026
7,8.876472473144531,0.45943022,3425.612248542568,14.973392009735107,0.5951239,3426.166809922754,26.074223518371582,0.8691399,3426.4977350764207,28.75680637359619,1.0274451,3426.6005051182997,19.10169792175293,1.1063517,3426.28817467706,19.96549701690674,1.2101994,3426.3181103075926
8,8.583518505096436,0.46407843,3426.4011157305595,14.526179790496826,0.5859889,3426.9558048179247,22.295742988586426,0.83868134,3427.286806178697,22.90949058532715,1.0037756,3427.389599886977,23.002519607543945,1.1002498,3427.0771975207153,15.132983684539795,1.2054036,3427.1071400449755
